In [1]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 68 bytes


In [2]:
! kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 99% 80.0M/80.9M [00:04<00:00, 22.2MB/s]
100% 80.9M/80.9M [00:04<00:00, 17.2MB/s]


In [3]:
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'
with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print("The Dataset is Extracted")

The Dataset is Extracted


In [4]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [7]:
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1')

In [8]:
twitter_data.shape

(1599999, 6)

In [9]:
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [10]:
twitter_data.tail()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
1599994,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599995,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599996,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599997,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...
1599998,4,2193602129,Tue Jun 16 08:40:50 PDT 2009,NO_QUERY,RyanTrevMorris,happy #charitytuesday @theNSPCC @SparksCharity...


from matplotlib import pyplot as plt
_df_0['1467810369'].plot(kind='hist', bins=20, title='1467810369')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('Mon Apr 06 22:19:45 PDT 2009').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_2.groupby('_TheSpecialOne_').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

/usr/local/lib/python3.10/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 153 (\x99) missing from current font.
  plt.savefig(


...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_11['Mon Apr 06 22:19:45 PDT 2009'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_11, x='1467810369', y='Mon Apr 06 22:19:45 PDT 2009', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_12['_TheSpecialOne_'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_12, x='1467810369', y='_TheSpecialOne_', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

/usr/local/lib/python3.10/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 153 (\x99) missing from current font.
  plt.savefig(


from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_13["@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_13, x='1467810369', y="@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D", inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [11]:
column_name = ['target','id','date','flag','user','text']
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names = column_name,encoding = 'ISO-8859-1')

In [12]:
twitter_data.shape

(1600000, 6)

In [13]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [14]:
twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [15]:
twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [16]:
twitter_data.replace({'target':{4:1}},inplace=True)

In [17]:
twitter_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

0 --> Negative Tweet

1 --> Positive Tweet

**Stemming**

In [18]:
port_stem = PorterStemmer()

In [19]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word)for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)

  return stemmed_content

In [20]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [21]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [105]:
twitter_data.tail()

,target,id,date,flag,user,text,stemmed_content
1599995,1,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...,woke school best feel ever
1599996,1,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...,thewdb com cool hear old walt interview http b...
1599997,1,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...,readi mojo makeov ask detail
1599998,1,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...,happi th birthday boo alll time tupac amaru sh...
1599999,1,2193602129,Tue Jun 16 08:40:50 PDT 2009,NO_QUERY,RyanTrevMorris,happy #charitytuesday @theNSPCC @SparksCharity...,happi charitytuesday thenspcc sparkschar speak...


In [22]:
print(twitter_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [23]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [24]:
X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values

In [25]:
print(X)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [26]:
print(Y)

[0 0 0 ... 1 1 1]


Splitting The data to training data and testing data

In [27]:
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,stratify = Y,random_state = 4)

In [28]:
print(X.shape,x_train.shape,x_test.shape)

(1600000,) (1280000,) (320000,)


In [29]:
print(x_train)

['realli half minut ago sinc updat im xd aaah love shame cnt go letter tho'
 'tad depress realiti best week ever realli sink' 'joannahelm hear tgif'
 ... 'still hive might life waaa realli autoimmun' 'get readi'
 'sleepytim now wanna wait dad come home though mum drive crazi usual nightnight x']


In [30]:
print(x_test)

['jordanknight bozo clown' 'ricopremier took tylenol make drowsi'
 'sigh guess start pay' ...
 'twitbotnew avi kay would like appli assimov law robot twitter bot talk feel http bit ly jebuk'
 'need bigger ipod mb left delet mb worth make room babi darl doll face honey'
 'wine festiv today bore come drink draw merri']


In [31]:
vectorizer = TfidfVectorizer()

x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)

In [32]:
print(x_train)

  (0, 407122)	0.2449378566345004
  (0, 233063)	0.32899012562742846
  (0, 150078)	0.13596830703457027
  (0, 77292)	0.34872708210000086
  (0, 361978)	0.28708270745925357
  (0, 242291)	0.15564710957377614
  (0, 363)	0.3713917824436101
  (0, 448836)	0.27098307408623185
  (0, 178191)	0.17416151253892917
  (0, 426518)	0.2397690303816763
  (0, 369099)	0.23778120529817967
  (0, 6242)	0.2593366351316874
  (0, 267342)	0.253746288012446
  (0, 158389)	0.25865057585916873
  (0, 335197)	0.17539295254819243
  (1, 369392)	0.46684901713605176
  (1, 124750)	0.28490203501945915
  (1, 438496)	0.24862635890744317
  (1, 39596)	0.27427987547601074
  (1, 335146)	0.41058423560799007
  (1, 98223)	0.3574859511688892
  (1, 393814)	0.4672392358157062
  (1, 335197)	0.22169841205474725
  (2, 400456)	0.5169109837538088
  (2, 162250)	0.337186327683649
  :	:
  (1279996, 453701)	0.2766279169821965
  (1279996, 446341)	0.25784921506074365
  (1279996, 375869)	0.2995561917098937
  (1279997, 28220)	0.5903579970418673
  (1279

In [33]:
print(x_test)

  (0, 202636)	0.4381262113355623
  (0, 76620)	0.5292805525416695
  (0, 49732)	0.7265724462446476
  (1, 422342)	0.589130338879642
  (1, 413338)	0.39075409533158667
  (1, 249396)	0.2725491608112262
  (1, 109866)	0.6526512359095383
  (2, 382262)	0.4236127291210829
  (2, 367853)	0.5596958576057355
  (2, 311123)	0.5357663703607016
  (2, 155744)	0.4693050171222206
  (3, 445626)	0.3345637040340407
  (3, 438948)	0.1873983153969061
  (3, 435623)	0.19873137414154904
  (3, 358703)	0.1802954672017031
  (3, 337621)	0.2855099603443114
  (3, 275099)	0.5434217568369408
  (3, 169708)	0.18539253779459292
  (3, 168865)	0.1897781207255086
  (3, 150078)	0.14176162309909768
  (3, 120758)	0.26148906614865175
  (3, 74461)	0.28255440351217553
  (3, 60521)	0.22304891131105312
  (3, 39686)	0.3401152981740468
  (4, 445689)	0.257082503746696
  :	:
  (319997, 43111)	0.1749267591001244
  (319997, 28625)	0.3658375538604148
  (319997, 21511)	0.2879505858634176
  (319998, 446248)	0.22917000584333636
  (319998, 345411)	

Training the Machine Learning Model

Logistic Regression

In [34]:
model = LogisticRegression(max_iter=1000)

In [35]:
model.fit(x_train, y_train)

LogisticRegression(max_iter=1000)

Model Evaluation

Accuracy Score

In [43]:
 x_train_prediction = model.predict(x_train)
 training_data_accuracy= accuracy_score(y_train,x_train_prediction)

In [44]:
print('Accuracy Score on the Training Data: ',training_data_accuracy)

Accuracy Score on the Training Data:  0.810075


In [45]:
 x_test_prediction = model.predict(x_test)
 testing_data_accuracy= accuracy_score(y_test,x_test_prediction)

In [46]:
print('Accuracy Score on the Testing Data: ',testing_data_accuracy)

Accuracy Score on the Testing Data:  0.779421875


Naive Bayes

In [47]:
from sklearn.naive_bayes import MultinomialNB

In [48]:
nb = MultinomialNB()

In [49]:
nb.fit(x_train, y_train)

MultinomialNB()

In [50]:
y_pred = nb.predict(x_test)

In [51]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.755859375


Random Forest

In [52]:
# from sklearn.ensemble import RandomForestClassifier
# rf = RandomForestClassifier(n_estimators=100, random_state=42)
# rf.fit(x_train, y_train)
# y_pred = rf.predict(x_test)
# accuracy = accuracy_score(y_test, y_pred)
# print(f"Accuracy: {accuracy}")

In [53]:
import pickle
filename = 'trained_model.sav'
pickle.dump(model,open(filename,'wb'))

In [54]:
loaded_model = pickle.load(open('/content/trained_model.sav','rb'))

In [111]:
# Save the TF-IDF vectorizer and the model
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')
pickle.dump(model, open('logistic_regression_model.pkl', 'wb'))

In [55]:
x_new = x_test[200]
print(y_test[200])
pred = loaded_model.predict(x_new)
print(pred)
if(pred[0] == 0):
  print('Negative Tweet')
else:
  print("Positive Tweet")

1
[1]
Positive Tweet


In [58]:
x_new = x_test[5]
print(y_test[5])
pred = loaded_model.predict(x_new)
print(pred)
if(pred[0] == 0):
  print('Negative Tweet')
else:
  print("Positive Tweet")

1
[1]
Positive Tweet


In [110]:
# Save the trained model using joblib
import joblib

joblib.dump(model, 'logistic_regression_model.pkl')

# Load the saved model
loaded_model = joblib.load('logistic_regression_model.pkl')


In [92]:
!pip install nltk


In [95]:
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def preprocess_input(text):
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Tokenize the text
    tokens = word_tokenize(text.lower())

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Join tokens back into a single string
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [108]:
loaded_model = pickle.load(open(filename, 'rb'))

# Function to classify tweet
def classify_tweet(tweet):
    # Preprocess the user input
    preprocessed_tweet = preprocess_input(tweet)

    # Transform the preprocessed text using the loaded vectorizer
    tweet_tfidf = vectorizer.transform([preprocessed_tweet])

    # Predict the sentiment of the new text
    pred = loaded_model.predict(tweet_tfidf)

    # Print the prediction and sentiment
    if pred[0] == 0:
        return 'Negative Tweet'
    else:
        return 'Positive Tweet'

# Take input from the user
user_input = input("Enter a tweet: ")

# Classify the input tweet
result = classify_tweet(user_input)
print(result)

Enter a tweet: Are you ready for your MoJo Makeover? Ask me for details
Positive Tweet
